작성할 문서
- unet 생성
-텐서플로우 사용법
- 넘파이 사용법(무작위 배열 생성법, 넘파이 변환, 조작, 차원추)
-판다스, matplotlib사용법
-파이썬 코테 라이브러리


이미지 전처리

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

In [9]:
path="C:/Users/82109/Desktop/datasets"

#해당 디렉토리의 모든 파일명을 가져옴

gifPath=[]
#경로 반환
def getFilePath(path,end=""):
  gifFileList=os.listdir(path)
  gifPath=[]
  for name in gifFileList:
    if name.endswith(".gif"):
      gifPath.append(os.path.join(path,name))
  return gifPath


#못쓰는 데이터를 걸러줌
def preprocessGif(path,frame=5):
  gif=Image.open(path)
  if gif.n_frames<frame:
    print(path,": ",gif.n_frames,"사용불가능")
    os.remove(path=path)
  else:
    print(path,": ",gif.n_frames," 사용가능")


gifPath=getFilePath(path) 

for i in gifPath:
  preprocessGif(i)


C:/Users/82109/Desktop/datasets\0.gif :  4 사용불가능


PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:/Users/82109/Desktop/datasets\\0.gif'

In [8]:
#gif를 읽고 넘파이 배열로 노멀라이즈해줌
def loadGif(path):
  gif=Image.open(path)
  flip=np.random.randint(0,7,1)[0]
  
  images=[]
  for i in range(1,gif.n_frames):
    gif.seek(i)
    temp=gif.transpose(flip).convert("RGBA")
    temp=np.array(temp)
    images.append(temp)
  gif.close()
  return np.array(images)/255.0

#인풋데이터와 아웃풋 데이터를 분리
def divide(arr):
  evens=arr[0::2]
  odds=arr[1::2]
  if evens.shape[0] != odds.shape[0]:
    evens=evens[0:-1]
  return [evens,odds]


arr=loadGif(gifPath[6])
input, output=divide(arr)
print(arr.shape ,input.shape, output.shape)

(28, 480, 773, 4) (14, 480, 773, 4) (14, 480, 773, 4)


unet 모델 생성

In [270]:
a=np.random.random((1,3,4,4,4))
layerNorm=tf.keras.layers.LayerNormalization()(a)

[[[[[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]

   [[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]

   [[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]

   [[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]]


  [[[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]

   [[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]

   [[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]

   [[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]]


  [[[ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]
    [ 12. 123.  12. 123.]
    [ 10.  10.  10.  10.]]

   [[ 12. 123.  12. 123.]
    [ 10.  10.  

In [272]:
layerNorm

<tf.Tensor: shape=(1, 3, 4, 4, 4), dtype=float32, numpy=
array([[[[[-0.9999998,  0.9999998, -0.9999998,  0.9999998],
          [ 0.       ,  0.       ,  0.       ,  0.       ],
          [-0.9999998,  0.9999998, -0.9999998,  0.9999998],
          [ 0.       ,  0.       ,  0.       ,  0.       ]],

         [[-0.9999998,  0.9999998, -0.9999998,  0.9999998],
          [ 0.       ,  0.       ,  0.       ,  0.       ],
          [-0.9999998,  0.9999998, -0.9999998,  0.9999998],
          [ 0.       ,  0.       ,  0.       ,  0.       ]],

         [[-0.9999998,  0.9999998, -0.9999998,  0.9999998],
          [ 0.       ,  0.       ,  0.       ,  0.       ],
          [-0.9999998,  0.9999998, -0.9999998,  0.9999998],
          [ 0.       ,  0.       ,  0.       ,  0.       ]],

         [[-0.9999998,  0.9999998, -0.9999998,  0.9999998],
          [ 0.       ,  0.       ,  0.       ,  0.       ],
          [-0.9999998,  0.9999998, -0.9999998,  0.9999998],
          [ 0.       ,  0.       ,  0

In [ ]:
def block():
    tf.keras.
    
def unetModel():
    layerNorm=tf.keras.layers.LayerNormalization()



In [217]:
import tensorflow as tf
def unetModel(inputShape=(None, None, None, 4), depth=5):
  inputs = tf.keras.Input(shape=inputShape)
  inputs2 = tf.keras.Input(shape=(inputShape[0],inputShape[1],inputShape[2],1))
  x1=tf.keras.layers.Concatenate()([inputs ,inputs2])
  x1=tf.keras.layers.Conv3D(32, 3,  padding='same',name="1conv")(x1)
  x1=tf.keras.layers.BatchNormalization()(x1)
  x1=tf.keras.layers.Activation("swish")(x1)
  x1=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(x1)
  x1=tf.keras.layers.Conv3D(64, 3,  padding='same',name="2conv")(x1)
  x1=tf.keras.layers.BatchNormalization()(x1)
  x1=tf.keras.layers.Activation("swish")(x1)
  x1=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(x1)

  
  return tf.keras.Model([inputs,inputs2],x1)

TensorShape([None, 128, 128, 32])

In [213]:
model=unetModel(inputShape=(37,320,320,4))
model.compile(
    tf.keras.optimizers.experimental.AdamW(
        learning_rate=1e-3, weight_decay=1e-4
    ),
    loss=tf.keras.losses.mean_absolute_error,
              metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_80 (InputLayer)          [(None, 37, 320, 32  0           []                               
                                0, 4)]                                                            
                                                                                                  
 input_81 (InputLayer)          [(None, 37, 320, 32  0           []                               
                                0, 1)]                                                            
                                                                                                  
 concatenate_12 (Concatenate)   (None, 37, 320, 320  0           ['input_80[0][0]',               
                                , 5)                              'input_81[0][0]']         

<tf.Tensor: shape=(1, 37, 80, 80, 64), dtype=float32, numpy=
array([[[[[-3.48716676e-02,  5.87121435e-02,  3.41480561e-02, ...,
           -3.40322703e-02,  2.06439123e-02,  1.66877657e-02],
          [-4.00659628e-02,  2.34624594e-02,  6.99023679e-02, ...,
           -2.05721371e-02,  2.57768445e-02,  3.54886129e-02],
          [-4.48450483e-02,  2.32490562e-02,  3.74265909e-02, ...,
           -2.01759730e-02,  4.42298427e-02,  2.54382528e-02],
          ...,
          [-2.69818325e-02,  2.92972475e-03,  8.48119259e-02, ...,
           -1.38065722e-02,  4.71701287e-02,  3.29402201e-02],
          [-3.11148930e-02,  2.76899878e-02,  9.00047198e-02, ...,
           -9.99139249e-03,  4.04806100e-02,  4.46504839e-02],
          [-2.79864948e-03,  2.87749525e-02,  7.37812445e-02, ...,
            1.33217154e-02,  7.56012499e-02,  3.86752002e-02]],

         [[-8.15486982e-02,  4.95716706e-02,  7.44239390e-02, ...,
           -4.01449203e-02,  1.00009004e-02,  2.71059740e-02],
          [-

Unet예시

In [ ]:
#참고 unet
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate

def build_3d_unet(input_shape):
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    # ... Add more layers as needed
    
    # Decoder
    up1 = UpSampling3D(size=(2, 2, 2))(pool1)
    concat1 = concatenate([up1, conv1], axis=-1)
    conv2 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(concat1)
    # ... Add more layers as needed
    
    # Output layer
    output = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv2)
    
    model = tf.keras.Model(inputs=inputs, outputs=output)
    return model

# Example usage
input_shape = (None, None, None, 1)  # Set the dimensions as needed
model = build_3d_unet(input_shape)
model.summary()



In [ ]:
# 텐서플로우 임포트
import tensorflow as tf

# unet의 컨볼루션 블록 정의
def conv_block(inputs, filters, kernel_size=3, strides=1, padding='same', activation='swish', batch_norm=True):
  x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(inputs)
  if batch_norm:
    x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(activation)(x)
  return x

# unet의 컨볼루션 트랜스포즈 블록 정의
def conv_transpose_block(inputs, filters, kernel_size=3, strides=2, padding='same', activation='swish', batch_norm=True):
  x = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)(inputs)
  if batch_norm:
    x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation(activation)(x)
  return x

# unet의 컨캣네이트 레이어 정의
def concat_layer(inputs1, inputs2):
  return tf.keras.layers.Concatenate()([inputs1, inputs2])

# unet 모델 생성 함수 정의
def unet_model(input_shape=(None, None, None, 4), depth=5):
  # 입력 레이어 생성
  inputs = tf.keras.Input(shape=input_shape)

  # 입력값 shape가 (1) 인 순서 정보를 나타내는 정수 벡터 생성
  frames = input_shape[0] # 프레임 수
  order = tf.range(frames) # [0, 1, ..., frames-1] 벡터

  # 정수 벡터를 임베딩 레이어에 통과시켜 임베딩 벡터 생성
  embedding_dim = input_shape[-1] # 임베딩 차원은 입력 채널 수와 동일하게 설정
  embedding_layer = tf.keras.layers.Embedding(frames, embedding_dim) # 임베딩 레이어 생성
  embedding_vector = embedding_layer(order) # [frames, embedding_dim] 크기의 임베딩 벡터 생성

  # 임베딩 벡터를 입력 이미지와 동일한 높이와 가로 크기로 확장
  height = input_shape[1] # 입력 이미지의 높이
  width = input_shape[2] # 입력 이미지의 가로
  embedding_matrix = tf.reshape(embedding_vector, [frames, 1, 1, embedding_dim]) # [frames, 1, 1, embedding_dim] 크기로 변형
  embedding_matrix = tf.tile(embedding_matrix, [1, height, width, 1]) # [frames, height, width, embedding_dim] 크기로 타일링

  # 확장된 임베딩 행렬을 입력 이미지와 채널 차원에서 결합
  inputs_with_order = concat_layer(inputs, embedding_matrix) # [frames, height, width, input_shape[-1]+embedding_dim] 크기의 새로운 입력 이미지 생성

  # contracting path 생성
  skip_connections = [] # 컨캣네이트 레이어에 사용할 스킵 연결 저장할 리스트
  x = inputs_with_order # 새로운 입력 이미지를 첫 번째 입력으로 사용
  for i in range(depth):
    filters = 2**(i+6) # 각 단계마다 필터 수를 두 배씩 증가시킴 (64부터 시작)
    x = conv_block(x, filters) # 컨볼루션 블록 적용
    skip_connections.append(x) # 스킵 연결로 저장
    x = tf.keras.layers.MaxPool2D(2)(x) # 맥스풀링 레이어 적용

  # expansive path 생성
  for i in range(depth):
    filters = 2**(depth-i+5) # 각 단계마다 필터 수를 절반씩 감소시킴 (512부터 시작)
    x = conv_transpose_block(x, filters) # 컨볼루션 트랜스포즈 블록 적용
    x = concat_layer(x, skip_connections[-i-1]) # 컨캣네이트 레이어로 스킵 연결과 결합

  # 출력 레이어 생성
  outputs = conv_block(x, input_shape[-1], activation='sigmoid', batch_norm=False) # 입력 이미지와 동일한 채널 수의 출력 이미지 생성

  # 모델 객체 생성
  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  # 모델 반환
  return model

# unet 모델 생성 예시
model = unet_model(input_shape=(10, 256, 256, 4), depth=5) # 입력값의 형태가 (프레임, 높이, 가로, 채널) shape이 (10, 256, 256, 4)인 unet 모델 생성
model.summary() # 모델 요약 정보 출력